## Introduction 
In this question, you'll build a basic recommendation system using collaborative filtering to make recommendations on a typical recommendation systems dataset, the MovieLens dataset. The purpose of this question is to become familiar with the internals of recommendation systems: both how they train and how they form recommendations.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.linalg as la
%matplotlib inline
plt.rcParams['figure.figsize'] = 12,10

## Collaborative Filtering by Matrix Factorization
In collaborative filtering we wish to factorize our ratings matrix into two smaller feature matrices whose product is equal to the original ratings matrix. Specifically, given some partially filled ratings matrix $X\in \mathbb{R}^{m\times n}$, we want to find feature matrices $U \in \mathbb{R}^{m\times k}$ and $V \in \mathbb{R}^{n\times k}$ such that $UV^T = X$. In the case of movie recommendation, each row of $U$ could be features corresponding to a user, and each row of $V$ could be features corresponding to a movie, and so $u_i^Tv_j$ is the predicted rating of user $i$ on movie $j$. This forms the basis of our hypothesis function for collaborative filtering: 

$$h_\theta(i,j) = u_i^T v_j$$

In general, $X$ is only partially filled (and usually quite sparse), so we can indicate the non-presence of a rating with a 0. Notationally, let $S$ be the set of $(i,j)$ such that $X_{i,j} \neq 0$, so $S$ is the set of all pairs for which we have a rating. The loss used for collaborative filtering is squared loss:

$$\ell(h_\theta(i,j),X_{i,j}) = (h_\theta(i,j) - X_{i,j})^2$$

The last ingredient to collaborative filtering is to impose an $l_2$ weight penalty on the parameters, so our total loss is:

$$\sum_{i,j\in S}\ell(h_\theta(i,j),X_{i,j}) + \lambda_u ||U||_2^2 + \lambda_v ||V||_2^2$$

For this assignment, we'll let $\lambda_u = \lambda_v = \lambda$. 

## MovieLens rating dataset
To start off, let's get the MovieLens dataset. This dataset is actually quite large (24+ million ratings), but we will instead use their smaller subset of 100k ratings. You can go  fetch this from their website, or download from Canvas. 

* You can download the archive containing their latest dataset release from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip (last updated October 2018). 
* For more details (contents and structure of archive), you can read the README at http://files.grouplens.org/datasets/movielens/ml-latest-README.html
* You can find general information from their website description located at http://grouplens.org/datasets/movielens/. 

For this assignment, we will only be looking at the ratings data specifically. However, it is good to note that there is additional data available (i.e. movie data and user made tags for movies) which could be used to improve the ability of the recommendation system. 

## Reading the dataset

In [2]:
datadir = 'ml-latest-small/'
movies = pd.read_csv(datadir + "movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv(datadir + "ratings.csv")
print("There are %d unique users and %d unique movies in the dataset which have been watched by at least one user." 
      % (len(set(ratings.userId)),len(set(ratings.movieId))))
ratings.head()

There are 610 unique users and 9724 unique movies in the dataset which have been watched by at least one user.


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Data Cleaning and Data Preparation

- Some movies are not watched by any user, but are present in the movie dataset. We get the list of this movies so we can handle them correctly in our analysis.

- Matrix factorization requires that we have our ratings stored in a matrix of users, so the first task is to take the ratings dataframe and convert it into this format. Note that in general, these matrices are extremely large and sparse (especially if you want to process the 24 million ratings), however for the purposes of this assignment a dense representation of the smaller dataset we are using should fit on any machine.

- We pivot the ratings dataframe so that we have rows as userIds and columns as movieIds. Then, we append the unwatched movies as additional columns at the end of that pivoted table.

In [4]:
unwatched_movies = list(set(movies.movieId).difference(set(ratings.movieId)))
print("There are %d unwatched movies in the movie dataset." %len(unwatched_movies))

There are 18 unwatched movies in the movie dataset.


In [5]:
ratings_table = ratings.pivot(index='userId',columns='movieId',values='rating').fillna(0)
unwatched_df = pd.DataFrame(np.zeros((ratings_table.shape[0],len(unwatched_movies))),
                            index=ratings_table.index,columns=unwatched_movies)
ratings_table = pd.concat([ratings_table,unwatched_df],axis=1)
ratings_table

,1,2,3,4,5,6,7,8,9,10,...,7020,30892,7792,34482,32371,1076,5721,2939,8765,25855
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Dividing data into a training and test set

- Make two copies of the ratings_table: one for training and one for testing.
- Select $\mathrm{floor}(9n/10)$ of the columns of (copy of the) ratings_table (where $n$ is the number of movies) to be the training set. Zero out the remaining $\mathrm{floor}(n/10)$ of the columns -- this modified version of ratings_table will be the training set.
- The test set will have the same set of users as the training set, but non-zero values for the columns that we have not trained on.

In [6]:
X_tr = ratings_table.copy().values
X_te = ratings_table.copy().values

P = np.random.permutation(ratings_table.shape[1])
num_tr_cols = int(np.floor(ratings_table.shape[1]*0.9))

X_tr[:,P[num_tr_cols:]] = 0
X_te[:,P[:num_tr_cols]] = 0

## Map movieIds to titles

In [7]:
movie_names_df = pd.merge(movies,pd.DataFrame(list(ratings_table.columns),columns=['movieId']),on='movieId')[['movieId','title']]
movie_names_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


## Matrix factorization
We are going to consider a slightly different approach to collaborative filtering, which fits better into the machine learning frameworks we have discussed so far. For the user-user and item-item approaches, we didn’t find the notion of a hypothesis function, loss function, and optimization procedure, since there was really no training phase of the algorithm to speak of (the methods can be viewed in this way, with the weighting functions corresponding to different losses, but it’s not particularly intuitive). But with matrix factorization, the correspondence is much more direct, so we’ll use this notation here. The presentation here is somewhat brief.
The core idea of the matrix factorization approach to collaborative filtering is that we’re going to approximate the $X$ matrix as a product of two lower-rank matrices

$$ \hat{X} \approx U V, U \in \Re^{m\times k}, V \in \Re^{k\times n} $$ 
where $k≪m,n$. In the collaborative filtering setting we only look to approximate the matrix on terms that are observed in $X$.

Let’s define this a bit more formally in terms of the specific elements rows and columns of the matrices $U$ and $V$ respectively. For each user $i=1,\ldots,m$ we define a set of user-specific weights, $u_i \in \Re^k$; and for each item $j=1,\ldots, n$ we define a set of item-specific weights, $v_j \in \Re^k$. Our prediction $\hat{X}_{ij}$ (which we’ll more formally denote as our hypothesis function now), is simply given by

$$ \hat{X}_{ij} = h_{\theta}(i,j)= u_i^T  v_j $$
where here our parameters are just all the $u$ and $v$ vectors, $\theta=\{u_{1:m},v_{1:n}\}$. You can also think of $u_i$ and $v_j$ as being something that is both like a feature vector and a parameter vector. For a given user $i$, our hypothesis function is a linear hypothesis with parameters $u_i$, and we make our predictions by taking the inner product with these parameters and the item “features” $v_j$. Thus, the goal of matrix factorization is to simultaneously learn both the per-user coefficients and the per-item features.

To measure the performance of a given set of parameters, we’ll use the squared loss function,

$$ loss(h_{\theta}(i,j),X_{ij})={(h_{\theta}(i,j)−X_{ij})}^2 $$
but evaluated only for those entries where we have actually observed a rating in $X$. That is, defining $S$ as the set of observed user/item ratings

$$S=\{(i,j):X_{ij} \neq 0\} $$
our optimization problem is to find parameters that minimize

$$ minimize_{u_{1:m},v_{1:n}} \sum_{(i,j) \in S} {(u_i^T v_j−X_{ij})}^2 $$

We normally divide this loss by the number of non-zero entries in $X$, i.e., the size of $S$.

The regularized version of this loss is:

$$ minimize_{u_{1:m},v_{1:n}} \sum_{(i,j) \in S} {(u_i^T v_j−X_{ij})}^2 + \lambda \left(\sum_i {u_i}^T {u_i} + \sum_{j} {v_j}^T {v_j}\right)$$

## Alternating least squares
Finally, how do we solve this optimization problem? There are actually a number of ways of doing so (including traditional gradient descent), but a common method in practice here is to use a technique called alternating least squares. The method is common enough that it’s worth explaining in a bit of detail. The basic idea here is that it’s difficult to solve the optimization problem globally over both $u$ and $v$ (in fact this is a non-convex problem, so there is the possibility of local optima). 

$$ J_{u_{1:m},v_{1:n}}  = \sum_{(i,j) \in S} {(u_i^T v_j−X_{ij})}^2 + \lambda \left(\sum_i {u_i}^T {u_i} + \sum_{j} {v_j}^T {v_j}\right) $$

$$ minimize_{u_{1:m},v_{1:n}} J_{u_{1:m},v_{1:n}}$$

If we fix all the $v_j$ terms then we can find the optimal $u_i$ terms using simple least-squares; and similarly, if we fix all the $u_i$ terms we can solve globally for $v_j$. That is, we solve the equations in turn:


$$ \frac{\partial J_{u_{1:m},v_{1:n}}}{u_i} = 0$$

$$ \frac{\partial J_{u_{1:m},v_{1:n}}}{v_j} = 0$$


Our approach to solving the problem: start with some initial values of $u_i$ and $v_j$ (these could even be random), alternatively solve for all the $u_i$ then all $v_j$ terms with the equations above. 

These updates are each simple least-squares problems (so they have an anlytical solution). We hold $V$ fixed and calculate each component $U_i$ of $U$, $i = 1,\ldots, m$.

- we select $V_{:,idx}$ where $idx$ are the items $j$ that user $i$ has selected, i.e. $j$ such that $X_{ij} \neq 0$.
- we select $X_{i,idx}$ where $idx$ are the indices $j$ where $X_{ij} \neq 0$.
- we solve for $U_i$ using ${(V_{:,j} V_{:,j}^T)} U_i = V_{:,j}X_{i,idx}$ (this can be done with a call to np.linalg.solve)

Then, we hold $U$ fixed, and update $V_j$ for $j = 1,\ldots, n$ using the same idea as above.

- we select $U_{idx,:}$ where $idx$ are the users $i$ that have selected $j$, i.e. $i$ such that $X_{ij} \neq 0$.
- we select $X_{idx,:}$ where $idx$ are the users $i$ where $X_{ij} \neq 0$.
- we solve for $V_j$ using ${(U_{idx,:}^T U_{idx,:})} V_j = U_{idx,:}X_{idx,j}$ (this can be done with a call to np.linalg.solve)


## Compute the loss function 
- Given X, U, V, reg (lambda parameter)
- find indices i,j where $X_{ij} \neq 0$
- vectorize the loss computation

$$\frac{1}{|S|} \left[\sum_{(i,j): X_{ij} \neq 0} {(u_i . v_j - X_{ij})}^2 + \lambda \left(\sum_{i} u_i . u_i + \sum_{j} v_j . v_j \right)\right]$$

where $|S|$ is the number of non-zero entries in $X$.

In [8]:
def error(X, U, V,reg):
    """ Compute the mean error of the observed ratings in X and their estimated values. 
        Args: 
            X (numpy 2D array) : a ratings matrix as specified above 
            U (numpy 2D array) : a matrix of features for each user (m x k)
            V (numpy 2D array) : a matrix of features for each movie (k x n)
            reg (float): regularization parameter
        Returns: 
            (float) : the mean squared error of the observed ratings with their estimated values with penalty for u and v coefficients
        """
    uv = np.dot(U,V)
    diff = (uv - X)
    S = X != 0
    left = np.sum(diff[S] ** 2)
    
    u2 = np.sum(U ** 2)
    v2 = np.sum(V ** 2)
    return (left + (reg * (u2 + v2))) / (np.sum(S))
    


## Test the loss function
- set up random U and V matrices
- set $k$ to be the desired value. 
- set regularization parameter reg.
- call error(X,U,V,reg)

In [ ]:
X=X_tr

k = 10
U = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(X.shape[0],k)))
V = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(k,X.shape[1])))
reg = 10.0
error(X,U,V,reg)

## Train the model: build U and V
- Inputs: 
   - X_tr: training matrix
   - X_te: test matrix
   - k: number of latent features
   - U, V: matrix factors (randomly initialized)
   - niters: number of iterations to run
   - reg: regularization parameter
   - verbose: boolean flag to print diagnostics
- Output:
    U,V: user and item factors

In [9]:
def train(X_tr, X_te, k, U, V, niters=51, reg=10, verbose=False):
    """ Train a collaborative filtering model. 
        Args: 
            X_tr (numpy 2D array) : the training ratings matrix as specified above
            X_te (numpy 2D array) : the testing ratings matrix as specified above
            k (int) : the number of features used in the CF model
            U (numpy 2D array) : an initial matrix of features for each user
            V (numpy 2D array) : an initial matrix of features for each movie
            niters (int) : number of iterations to run
            reg (float) : regularization parameter
            verbose (boolean) : verbosity flag for printing useful messages
            
        Returns:
            (U,V) : A pair of the resulting learned matrix factorization
    """
    eye = reg * np.eye(k,k)
    
    notzero = X_tr > 0
    for obj1 in range(1, niters):
        for idx1 in range(X.shape[0]):
            loc = notzero[idx1,:]
            var1 = V[:,loc]
            var2 = np.dot(var1, X[idx1, loc])
            U[idx1,:] = np.linalg.solve(np.dot(var1,np.transpose(var1)) + eye, var2)
        for idx2 in range(X.shape[1]):
            loc = notzero[:,idx2]
            var1 = U[loc,:]
            var2 = np.dot(np.transpose(U[loc,]), X[loc, idx2])
            V[:,idx2] = np.linalg.solve(np.dot(np.transpose(var1), var1) + eye, var2)
        train = error(X,U,V,reg)
        test = error(X_te,U,V,0.0)
        print("Training Loss = " + str(train))
        print("Test Loss = " + str(test))
       
    return U,V

## Test your training function

In [ ]:
k = 20
U = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(X.shape[0],k)))
V = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(k,X.shape[1])))

U,V = train(X_tr, X_te, k, U, V, niters=51, reg=1, verbose=False)

## Predict the movie preference of user with U and V

- compute $U[i,:] V$ and argsort the $n$ ratings produced.
- use the movie_to_title dictionary to print out the names of the top 10 movies

In [10]:
def predict(U,V,user):
    """ Predict ratings for a user with a collaborative filtering model. 
        Args: 
            U (numpy 2D array) : a matrix of features for each user
            V (numpy 2D array) : a matrix of features for each movie
            user: id of user 
            
        Returns:
            A list of the top 10 movie titles preferred by the user. 
    """
    ratings=np.dot(U[user,:],V)
    top10=np.argsort(ratings)[-10:]
    names = [movie_names_df.to_dict('index')[obj]['title'] for obj in top10]
    return names

In [ ]:
predict(U,V,2)

## Obtain the top 10 choices of user from X

In [11]:
def top10(X,user):
    """ Read the top 10 choices for user from input matrix X. 
        Args: 
            X (numpy 2D array) : ratings matrix
            user: id of user 
            
        Returns:
            A list of the top 10 movie titles actually watched by user. 
    """    
    ratings=X[user,:]
    top10=np.argsort(ratings)[-10:]
    names = [movie_names_df.to_dict('index')[obj]['title'] for obj in top10]
    return names

In [ ]:
top10(X,2)